# Semantic Kernel na may Integrasyon ng OpenBnB MCP Server

Ipinapakita ng notebook na ito kung paano gamitin ang Semantic Kernel kasama ang aktwal na OpenBnB MCP server upang maghanap ng totoong mga tirahan sa Airbnb gamit ang MCPStdioPlugin. Para sa LLM Access, gumagamit ito ng Microsoft Foundry. Upang i-setup ang iyong mga environment variables, maaari mong sundan ang [Setup Lesson ](/00-course-setup/README.md)


## I-import ang Mga Kailangan na Package


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Paglikha ng MCP Plugin Connection

Ikokonekta namin sa [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) gamit ang MCPStdioPlugin. Ang server na ito ay nagbibigay ng Airbnb search functionality sa pamamagitan ng @openbnb/mcp-server-airbnb package.


## Paglikha ng Kliyente

Sa halimbawa na ito, gagamitin namin ang Microsoft Foundry para sa aming access sa LLM. Siguraduhing maayos ang pagkakaayos ng iyong mga environment variable.


## Environment Configuration

I-configure ang mga setting ng Azure OpenAI. Siguraduhing mayroon kang mga sumusunod na environment variable na nakaset:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Pag-unawa sa OpenBnB MCP Integration

Ang notebook na ito ay kumokonekta sa **tunay na OpenBnB MCP server** na nagbibigay ng aktwal na Airbnb search functionality.

### Paano ito gumagana:

1. **MCPStdioPlugin**: Gumagamit ng standard input/output na komunikasyon sa MCP server
2. **Tunay na NPM Package**: Dinadownload at pinapatakbo ang `@openbnb/mcp-server-airbnb` gamit ang npx
3. **Live Data**: Nagbabalik ng aktwal na data ng Airbnb properties mula sa kanilang mga API
4. **Function Discovery**: Awtomatikong nadidiskubre ng agent ang mga available na functions mula sa MCP server

### Mga Available na Function:

Karaniwang ibinibigay ng OpenBnB MCP server ang:
- **search_listings** - Maghanap ng Airbnb properties ayon sa lokasyon at mga pamantayan
- **get_listing_details** - Kunin ang detalyadong impormasyon tungkol sa mga partikular na properties
- **check_availability** - Suriin ang availability para sa mga partikular na petsa
- **get_reviews** - Kunin ang mga review para sa mga properties
- **get_host_info** - Kunin ang impormasyon tungkol sa mga host ng property

### Mga Kinakailangan:

- **Node.js** na naka-install sa iyong sistema
- **Internet connection** para i-download ang MCP server package
- **NPX** na available (kasama sa Node.js)

### Pagsubok ng Koneksyon:

Maaari mong subukan ang MCP server nang manu-mano sa pamamagitan ng pagpapatakbo ng:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Idadownload at sisimulan nito ang OpenBnB MCP server, na ikokonekta ng Semantic Kernel para sa tunay na data ng Airbnb.


## Pagpapatakbo ng Agent gamit ang OpenBnB MCP Server

Ngayon ay patatakbuhin natin ang AI Agent na kumokonekta sa OpenBnB MCP server upang maghanap ng totoong mga Airbnb na tirahan sa Stockholm para sa 2 matatanda at 1 bata. Malaya kang baguhin ang listahan ng `user_inputs` upang i-modify ang mga kriteriya sa paghahanap.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Buod
Malugod na pagbati! Matagumpay mong nabuo ang isang AI agent na nagsasama sa paghahanap ng tirahan sa totoong mundo gamit ang Model Context Protocol (MCP):

## Mga Teknolohiyang Ginamit:
- Semantic Kernel - Para sa paggawa ng matatalinong agent gamit ang Azure OpenAI
- Microsoft Foundry - Para sa kakayahan ng LLM at kompletong chat
- MCP (Model Context Protocol) - Para sa standardisadong pagsasama ng mga tool
- OpenBnB MCP Server - Para sa totoong Airbnb na pag-andar ng paghahanap
- Node.js/NPX - Para sa pagpapatakbo ng panlabas na MCP server

## Mga Natutunan Mo:
- MCP Integration: Pagsasama ng mga Semantic Kernel agent sa panlabas na MCP server
- Real-time Data Access: Paghahanap ng tunay na mga ari-arian sa Airbnb gamit ang live na API
- Protocol Communication: Paggamit ng stdio na komunikasyon sa pagitan ng agent at MCP server
- Function Discovery: Awtomatikong pagtuklas ng mga magagamit na function mula sa MCP server
- Streaming Responses: Pagkuha at pag-log ng tawag sa mga function nang real-time
- HTML Rendering: Pag-format ng mga sagot ng agent gamit ang mga naka-istilong talahanayan at interactive na pagpapakita

## Mga Susunod na Hakbang:
- Isama ang karagdagang MCP server (panahon, mga flight, mga restawran)
- Bumuo ng multi-agent system na pinagsasama ang MCP at A2A protocol
- Lumikha ng sariling MCP server para sa iyong sariling mga pinagkukunan ng datos
- Ipatupad ang permanenteng memorya ng usapan sa iba’t ibang session
- I-deploy ang agent sa Azure Functions kasama ang MCP server orchestration
- Magdagdag ng user authentication at kakayahan sa booking


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagamat nagsusumikap kami para sa katumpakan, pakatandaan na ang mga awtomatikong salin ay maaaring maglaman ng mga pagkakamali o di-tiyak na impormasyon. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na pangunahing sanggunian. Para sa mga mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng salin na ito.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
